In [89]:
import backtrader as bt
import matplotlib.pyplot as plt
from datetime import datetime

In [91]:
import pandas as pd
import numpy as np

# Define the file name
file_name = 'UoB_Set01_2025-01-02.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_name)

# Convert the 'Time' column to string to ensure proper formatting
df['Time'] = df['Time'].astype(str)

# Correct the date concatenation: the date should be a string
df['Datetime'] = pd.to_datetime('2025-01-02' + ' ' + df['Time'])

# Set 'Datetime' as the index
df.set_index('Datetime', inplace=True)

# Select only the numeric columns for resampling
numeric_data = df.select_dtypes(include=[np.number])

# Resample the numeric data to every minute, calculating the mean
df_resampled = numeric_data.resample('1T').mean()

# Display the first few rows to verify
print(df_resampled.head())

# Save the resampled DataFrame to a new CSV file
df_resampled.to_csv('UoB_Set01_2025-01-02_dt.csv')



                     Bid1_Price  Bid1_Volume  Ask1_Price  Ask1_Volume  \
Datetime                                                                
2025-01-02 07:30:00  261.840893    60.428571  270.011964    38.517857   
2025-01-02 07:31:00  258.679231    46.230769  263.964808    31.326923   
2025-01-02 07:32:00  263.103167    43.333333  267.750167    35.300000   
2025-01-02 07:33:00  263.017333    33.433333  267.201667    35.266667   
2025-01-02 07:34:00  263.175000    56.050000  268.676333    41.083333   

                     Bid2_Price  Bid2_Volume  Ask2_Price  Ask2_Volume  \
Datetime                                                                
2025-01-02 07:30:00  254.583214    48.589286  274.216964    39.839286   
2025-01-02 07:31:00  254.827885    47.692308  283.130192    33.807692   
2025-01-02 07:32:00  260.417000    53.650000  271.332667    28.683333   
2025-01-02 07:33:00  260.786500    48.616667  269.988333    32.983333   
2025-01-02 07:34:00  260.355167    64.650000  273.

Strategy Class

In [93]:
class CustomCSVData(bt.feeds.GenericCSVData):
    lines = ('mid_price', 'order_imbalance',)
    
    params = (
        #('fromdate', datetime(2025, 1, 2)),
        #('todate', datetime(2025, 1, 2)),
        #('nullvalue', 0.0),
        
        ('datetime', 0),  # We'll need to handle datetime externally if it's not in a standard column
        ('dtformat', ('%Y-%m-%d %H:%M:%S')),        

        # Adjusted mappings
        ('mid_price', 9),  # Index for Mid-Price
        ('volume', 18),  # Placeholder, assuming total volume isn't directly relevant
        #('openinterest', -1),  # Not typically relevant for LOB data
        
        # Additional fields
        ('order_imbalance', 19),  # Correct index for Order_imbalance
    )

class LOBStrategy(bt.Strategy):
    params = (
        ('imbalance_threshold', 0.2),  # Threshold for considering significant order imbalance
        ('printlog', True),            # Enable logging
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')

    def __init__(self):
        # Direct access to the 'mid' and 'order_imbalance' fields from the data feed
        self.mid_price = self.datas[0].mid_price
        self.order_imbalance = self.datas[0].order_imbalance

    def next(self):
        # Example logic based on the order imbalance and mid-price movement
        if not self.position:  # Not currently in the market
            if self.order_imbalance[0] > self.params.imbalance_threshold and self.mid_price[0] > self.mid_price[-1]:
                self.buy()
                self.log(f'BUY CREATE, {self.mid_price[0]}')
        else:
            if self.order_imbalance[0] < -self.params.imbalance_threshold and self.mid_price[0] < self.mid_price[-1]:
                self.sell()
                self.log(f'SELL CREATE, {self.mid_price[0]}')
                
# Create a Cerebro entity
cerebro = bt.Cerebro()

# Add the Data Feed to Cerebro
data = CustomCSVData(dataname='UoB_Set01_2025-01-02_dt.csv')
cerebro.adddata(data)

# Add the strategy to Cerebro
cerebro.addstrategy(LOBStrategy)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Run the strategy
cerebro.run()

# Plot the result (optional)
cerebro.plot()

2025-01-02, BUY CREATE, 281.05263157894734
2025-01-02, BUY CREATE, 281.51906976744186
2025-01-02, BUY CREATE, 266.3261016949152
2025-01-02, BUY CREATE, 280.75432432432433
2025-01-02, BUY CREATE, 266.7142424242424
2025-01-02, BUY CREATE, 265.185
2025-01-02, BUY CREATE, 263.94866666666667
2025-01-02, BUY CREATE, 262.47934782608695
2025-01-02, BUY CREATE, 266.371320754717
2025-01-02, BUY CREATE, 268.93285714285713
2025-01-02, BUY CREATE, 272.7366101694915
2025-01-02, BUY CREATE, 270.9173333333333
2025-01-02, BUY CREATE, 273.78543859649125
2025-01-02, BUY CREATE, 271.50883333333337
2025-01-02, BUY CREATE, 279.0316
2025-01-02, BUY CREATE, 276.57358490566037
2025-01-02, BUY CREATE, 278.63933333333335
2025-01-02, BUY CREATE, 288.0448780487805
2025-01-02, BUY CREATE, 286.79366666666664
2025-01-02, BUY CREATE, 283.7478333333333
2025-01-02, BUY CREATE, 286.96416666666664
2025-01-02, BUY CREATE, 289.02750000000003
2025-01-02, BUY CREATE, 287.3321052631579
2025-01-02, BUY CREATE, 288.8504999999999

MemoryError: Unable to allocate 353. TiB for an array with shape (48513168000003,) and data type float64